Finetune BERT models

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.adamw import AdamW

from datasets import Dataset, DatasetDict
from transformers import AutoModel, AutoTokenizer

from onnxruntime import InferenceSession

import duckdb

/homes/ukumaras/bin/miniconda3/envs/mldb/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load iris data

In [2]:
con = duckdb.connect("../imdb.db")
imdb = con.sql("SELECT * FROM imdb").df()
con.close()

imdb['label'] = imdb['label'].astype(int)
imdb.head()

,text,label,stage
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,train
1,"""I Am Curious: Yellow"" is a risible and preten...",0,train
2,If only to avoid making this type of film in t...,0,train
3,This film was probably inspired by Godard's Ma...,0,train
4,"Oh, brother...after hearing about this ridicul...",0,train


In [3]:
imdb_test = Dataset.from_pandas(imdb[imdb['stage'] == 'test'].drop(columns=['stage']))
imdb_train = Dataset.from_pandas(imdb[imdb['stage'] == 'train'].drop(columns=['stage']))

dataset = DatasetDict()
dataset['train'] = imdb_train
dataset['test'] = imdb_test

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 25000
    })
})

### Train the model

In [4]:
model_name = 'bert-tiny'
model_path = f"/homes/ukumaras/scratch/Models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_path)
bert_model = AutoModel.from_pretrained(model_path)

In [5]:
def preprocess(batch):
    return tokenizer(batch['text'], return_tensors="pt", padding="max_length", truncation=True, max_length=512)

dataset_encoded = dataset.map(preprocess, batched=True, batch_size=None)
# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True)


# tokenized_imdb = imdb.map(preprocess_function, batched=True)
# tokenized_imdb

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map: 100%|██████████| 25000/25000 [00:20<00:00, 1213.18 examples/s]


In [6]:
# bert_model

### Text Classification

In [7]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(model_path, num_labels=num_labels))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /homes/ukumaras/scratch/Models/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [9]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(dataset_encoded["train"])
model_name = f"{model_name}-finetuned"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error")

/homes/ukumaras/bin/miniconda3/envs/mldb/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["test"],
                  tokenizer=tokenizer)

In [11]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.531839,0.753360,0.752580
2,No log,0.422355,0.815720,0.815278
3,No log,0.381090,0.836800,0.836700
4,No log,0.371245,0.841200,0.840959
5,No log,0.363759,0.846040,0.845949


TrainOutput(global_step=1955, training_loss=0.44344279491687977, metrics={'train_runtime': 7775.6458, 'train_samples_per_second': 16.076, 'train_steps_per_second': 0.251, 'total_flos': 158810880000000.0, 'train_loss': 0.44344279491687977, 'epoch': 5.0})

In [12]:
trainer.save_model(model_path + '-imdb-cls')

In [14]:
dummy_model_input = tokenizer("This is a sample", return_tensors="pt")

In [18]:
torch.onnx.export(
    model, 
    tuple(dummy_model_input.values()),
    f=model_path+"/model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=19, 
)

/homes/ukumaras/bin/miniconda3/envs/mldb/lib/python3.12/site-packages/torch/onnx/utils.py:1547: OnnxExporterWarning: Exporting to ONNX opset version 19 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 17. To use a newer opset version, consider 'torch.onnx.dynamo_export()'. Note that dynamo_export() is in preview. Please report errors with dynamo_export() as Github issues to https://github.com/pytorch/pytorch/issues.
  warnings.warn(


In [13]:
fail

NameError: name 'fail' is not defined

In [ ]:
onnx_model_path = "/homes/ukumaras/scratch/Models/distilbert-base-uncased-finetuned-sst-2-english-onnx"

tokenizer = AutoTokenizer.from_pretrained(onnx_model_path)
session = InferenceSession(onnx_model_path + "/model.onnx")

In [ ]:

inputs = tokenizer("Using DistilBERT with ONNX Runtime!", return_tensors="np")
inputs

In [ ]:

outputs = session.run(output_names=["logits"], input_feed=dict(inputs))

In [ ]:
outputs

In [ ]:
from transformers.models.distilbert import DistilBertConfig, DistilBertOnnxConfig

config = DistilBertConfig()
onnx_config = DistilBertOnnxConfig(config)
print(list(onnx_config.outputs.keys()))

In [ ]:
onnx_config.inputs